In [1]:
from nba_api.stats.endpoints import SynergyPlayTypes, LeagueDashPtStats, CommonPlayerInfo, CommonTeamYears , CommonTeamRoster, CommonAllPlayers
from supabase import create_client, Client
import pandas as pd
import numpy as np
import os
import time

In [ ]:


supabase = create_client(SUPABASE_URL, SUPABASE_KEY)

In [3]:
# List of seasons from 2015-16 to 2024-25
seasons = [
    "2015-16", "2016-17", "2017-18", "2018-19", "2019-20",
    "2020-21", "2021-22", "2022-23", "2023-24", "2024-25"
]


all_players = pd.DataFrame()
seen_ids = set()

for season in seasons:
    print(f"Pulling players from season {season}")
    response = CommonAllPlayers(is_only_current_season=0, league_id="00", season=season)
    df = response.get_data_frames()[0]

    # Filter only players with ROSTERSTATUS == 1 (currently rostered)
    df = df[df["ROSTERSTATUS"] == 1]

    # Only add players not yet seen
    df = df[~df["PERSON_ID"].isin(seen_ids)]
    seen_ids.update(df["PERSON_ID"].tolist())

    all_players = pd.concat([all_players, df], ignore_index=True)
    time.sleep(1)

print(f"\n✅ Collected {len(all_players)} unique players")

print(all_players)


Pulling players from season 2015-16
Pulling players from season 2016-17
Pulling players from season 2017-18
Pulling players from season 2018-19
Pulling players from season 2019-20
Pulling players from season 2020-21
Pulling players from season 2021-22
Pulling players from season 2022-23
Pulling players from season 2023-24
Pulling players from season 2024-25

✅ Collected 1333 unique players
      PERSON_ID DISPLAY_LAST_COMMA_FIRST DISPLAY_FIRST_LAST  ROSTERSTATUS  \
0        203112              Acy, Quincy         Quincy Acy             1   
1        203919            Adams, Jordan       Jordan Adams             1   
2        203500            Adams, Steven       Steven Adams             1   
3        201167           Afflalo, Arron      Arron Afflalo             1   
4        201582           Ajinca, Alexis      Alexis Ajinca             1   
...         ...                      ...                ...           ...   
1328    1641817            Watson, Anton       Anton Watson         

In [ ]:
'''
# Create a list of unique player IDs
player_ids = all_players["PERSON_ID"].tolist()

detailed_players = []
failed_ids = []
consecutive_failures = 0

for pid in player_ids:
    success = False  # track if we succeed within 3 tries

    for attempt in range(3):
        try:
            info = CommonPlayerInfo(league_id_nullable="00", player_id=pid).get_data_frames()[0]

            row = {
                "player_id": str(info.loc[0, "PERSON_ID"]),
                "player_name": info.loc[0, "DISPLAY_FIRST_LAST"],
                "position": info.loc[0, "POSITION"],
                "height": info.loc[0, "HEIGHT"],
                "weight": info.loc[0, "WEIGHT"],
                "school": info.loc[0, "SCHOOL"],
                "birthdate": info.loc[0, "BIRTHDATE"],
                "draft_year": info.loc[0, "DRAFT_YEAR"],
                "draft_round": info.loc[0, "DRAFT_ROUND"],
                "draft_number": info.loc[0, "DRAFT_NUMBER"]
            }

            detailed_players.append(row)
            success = True
            consecutive_failures = 0  # reset streak on success
            break  # exit retry loop on success

        except Exception as e:
            print(f"Attempt {attempt + 1} failed for {pid}: {e}")
            time.sleep(30 + attempt)

    if not success:
        failed_ids.append(pid)
        consecutive_failures += 1

    if consecutive_failures >= 2:
        print("\n Two players in a row failed completely. Exiting loop early.")
        break

    time.sleep(5)  # base delay between each player fetch

# Convert to DataFrame
player_info_df = pd.DataFrame(detailed_players)

print(player_info_df)
'''

Attempt 1 failed for 203920: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)
Attempt 2 failed for 203920: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)
Attempt 3 failed for 203920: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)
Attempt 1 failed for 1628444: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)
Attempt 2 failed for 1628444: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)
Attempt 3 failed for 1628444: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)

 Two players in a row failed completely. Exiting loop early.


In [ ]:
'''
player_info_df.to_csv("player_info_full.csv", index=False)

'''

In [ ]:
'''

done_ids = set(player_info_df["player_id"])
new_player_ids = [pid for pid in all_players["PERSON_ID"].tolist() if str(pid) not in done_ids]

detailed_players_continued = []
failed_ids = []
consecutive_failures = 0

for pid in new_player_ids:
    success = False  # track if we succeed within 3 tries

    for attempt in range(3):
        try:
            info = CommonPlayerInfo(league_id_nullable="00", player_id=pid).get_data_frames()[0]

            row = {
                "player_id": str(info.loc[0, "PERSON_ID"]),
                "player_name": info.loc[0, "DISPLAY_FIRST_LAST"],
                "position": info.loc[0, "POSITION"],
                "height": info.loc[0, "HEIGHT"],
                "weight": info.loc[0, "WEIGHT"],
                "school": info.loc[0, "SCHOOL"],
                "birthdate": info.loc[0, "BIRTHDATE"],
                "draft_year": info.loc[0, "DRAFT_YEAR"],
                "draft_round": info.loc[0, "DRAFT_ROUND"],
                "draft_number": info.loc[0, "DRAFT_NUMBER"]
            }

            detailed_players_continued.append(row)
            success = True
            consecutive_failures = 0  # reset after success
            break

        except Exception as e:
            print(f"Attempt {attempt + 1} failed for {pid}: {e}")
            time.sleep(30 + attempt)

    if not success:
        failed_ids.append(pid)
        consecutive_failures += 1

    if consecutive_failures >= 2:
        print("\n Two players in a row failed completely. Exiting loop early.")
        break

    time.sleep(5)

# Merge with earlier DataFrame
player_info_df = pd.concat([
    player_info_df,
    pd.DataFrame(detailed_players_continued)
], ignore_index=True)


'''

KeyboardInterrupt: 

In [ ]:
'''

# Player's Bio Supabase Transfer
player_info_df["player_id"] = pd.to_numeric(player_info_df["player_id"])
player_info_df = player_info_df.drop_duplicates(subset="player_id")

records = player_info_df.to_dict(orient="records")


chunk_size = 50
for i in range(0, len(records), chunk_size):
    chunk = records[i:i + chunk_size]
    try:
        supabase.table("players").insert(chunk).execute()
        print(f"Inserted chunk {i // chunk_size + 1}")
        time.sleep(1)
    except Exception as e:
        print(f"Error inserting chunk {i // chunk_size + 1}: {e}")

        
'''

Inserted chunk 1
Inserted chunk 2
Inserted chunk 3
Inserted chunk 4
Inserted chunk 5
Inserted chunk 6
Inserted chunk 7
Inserted chunk 8
Inserted chunk 9
Inserted chunk 10
Inserted chunk 11
Inserted chunk 12
Inserted chunk 13
Inserted chunk 14
Inserted chunk 15
Inserted chunk 16
Inserted chunk 17
Inserted chunk 18
Inserted chunk 19
Inserted chunk 20
Inserted chunk 21
Inserted chunk 22
Inserted chunk 23
Inserted chunk 24
Inserted chunk 25
Inserted chunk 26
Inserted chunk 27


In [3]:
nba_teams = CommonTeamYears(league_id='00').get_data_frames()[0]


active_nba_teams = nba_teams[nba_teams["MAX_YEAR"] == '2024']


team_ids = active_nba_teams["TEAM_ID"].tolist()

# List of seasons from 2015-16 to 2024-25
seasons = [
    "2015-16", "2016-17", "2017-18", "2018-19", "2019-20",
    "2020-21", "2021-22", "2022-23", "2023-24", "2024-25"
]

team_rosters = []
consecutive_failures = 0

for season in seasons:
    
    
    for team_id in team_ids:

        success = False
        
        for attempt in range(3):
            try:
                info = CommonTeamRoster(team_id=team_id, season=season).get_data_frames()[0]
                
                for _, row in info.iterrows():
                    player = {
                        "season": season,
                        "team_id": team_id,
                        "player_id": str(row["PLAYER_ID"]),
                        "player_name": row["PLAYER"],
                        "position": row["POSITION"],
                        "age": row["AGE"],
                        "experience": row["EXP"]
                    }
                    team_rosters.append(player)
                
                success = True
                consecutive_failures = 0  # reset failure streak
                break  # success, break retry loop

            except Exception as e:
                print(f"Attempt {attempt + 1} failed for team {team_id} in {season}: {e}")
                time.sleep(30 + attempt)

        if not success:
            consecutive_failures += 1
            if consecutive_failures >= 2:
                print("Two consecutive team failures — exiting loop early.")
                break
        
        time.sleep(5) 



# Convert to DataFrame
team_roster_df = pd.DataFrame(team_rosters)
print(f"Finished with {len(team_roster_df)} player-team-season records.")
print(team_roster_df)

KeyboardInterrupt: 

In [ ]:
done_team_seasons = set((r["team_id"], r["season"]) for r in team_rosters)

new_pairs = [
    (tid, s) for s in seasons for tid in team_ids
    if (tid, s) not in done_team_seasons
]

failed_pairs = []
consecutive_failures = 0

for team_id, season in new_pairs:
    success = False  # track if we succeed within 3 tries

    for attempt in range(3):
        try:
            response = CommonTeamRoster(team_id=team_id, season=season)
            info = response.get_data_frames()[0]

            for _, row in info.iterrows():
                player = {
                    "season": season,
                    "team_id": team_id,
                    "player_id": str(row["PLAYER_ID"]),
                    "player_name": row["PLAYER"],
                    "position": row["POSITION"],
                    "age": row["AGE"],
                    "experience": row["EXP"]
                }
                team_rosters.append(player)

            success = True
            consecutive_failures = 0  # reset after success
            break  # exit retry loop

        except Exception as e:
            print(f"Attempt {attempt + 1} failed for team {team_id} in {season}: {e}")
            time.sleep(30 + attempt)

    if not success:
        failed_pairs.append((team_id, season))
        consecutive_failures += 1

    if consecutive_failures >= 2:
        print("\n❌ Two team-season combos in a row failed. Exiting loop early.")
        break

    time.sleep(5)  # to avoid rate limit issues




team_roster_df = pd.concat([
    team_roster_df,  # existing DataFrame
    pd.DataFrame(team_rosters)  # new results
], ignore_index=True)

In [4]:
team_roster_df.to_csv("team_info_full.csv", index=False)

NameError: name 'team_roster_df' is not defined

In [3]:
team_roster_df = pd.read_csv("team_info_full.csv")
team_roster_df["player_id"] = pd.to_numeric(team_roster_df["player_id"], errors='coerce')
team_roster_df["team_id"] = pd.to_numeric(team_roster_df["team_id"], errors='coerce')
team_roster_df["season"] = team_roster_df["season"].astype(str)
team_roster_df = team_roster_df.drop_duplicates(subset=["season", "team_id", "player_id"])
print(team_roster_df)

       season     team_id  player_id       player_name position   age  \
0     2015-16  1610612737     201952       Jeff Teague        G  28.0   
1     2015-16  1610612737     200794      Paul Millsap        F  31.0   
2     2015-16  1610612737     203501  Tim Hardaway Jr.        G  24.0   
3     2015-16  1610612737     201168    Tiago Splitter      C-F  31.0   
4     2015-16  1610612737       2550      Kirk Hinrich        G  35.0   
...       ...         ...        ...               ...      ...   ...   
4989  2024-25  1610612766    1641706    Brandon Miller        F  22.0   
4990  2024-25  1610612766    1642354        KJ Simpson        G  22.0   
4991  2024-25  1610612766     203552        Seth Curry        G  34.0   
4992  2024-25  1610612766    1642275    Tidjane Salaün        F  19.0   
4993  2024-25  1610612766     201959        Taj Gibson        F  39.0   

     experience  
0             6  
1             9  
2             2  
3             5  
4            12  
...         ...

In [7]:
team_roster_df["experience"] = team_roster_df["experience"].replace("R", 0).astype(int)
team_roster_df = team_roster_df.drop_duplicates()
print(team_roster_df)

       season     team_id  player_id       player_name position   age  \
0     2015-16  1610612737     201952       Jeff Teague        G  28.0   
1     2015-16  1610612737     200794      Paul Millsap        F  31.0   
2     2015-16  1610612737     203501  Tim Hardaway Jr.        G  24.0   
3     2015-16  1610612737     201168    Tiago Splitter      C-F  31.0   
4     2015-16  1610612737       2550      Kirk Hinrich        G  35.0   
...       ...         ...        ...               ...      ...   ...   
4989  2024-25  1610612766    1641706    Brandon Miller        F  22.0   
4990  2024-25  1610612766    1642354        KJ Simpson        G  22.0   
4991  2024-25  1610612766     203552        Seth Curry        G  34.0   
4992  2024-25  1610612766    1642275    Tidjane Salaün        F  19.0   
4993  2024-25  1610612766     201959        Taj Gibson        F  39.0   

      experience  
0              6  
1              9  
2              2  
3              5  
4             12  
...      

In [ ]:

# Convert to record dicts
team_records = team_roster_df.to_dict(orient="records")

# Upload in chunks
chunk_size = 1000
for i in range(0, len(team_records), chunk_size):
    chunk = team_records[i:i + chunk_size]
    try:
        supabase.table("teams_players").insert(chunk).execute()
        print(f"Inserted chunk {i // chunk_size + 1}")
        time.sleep(1)
    except Exception as e:
        print(f"Error inserting chunk {i // chunk_size + 1}: {e}")


Inserted chunk 1
Inserted chunk 2
Inserted chunk 3
Inserted chunk 4
Inserted chunk 5
Inserted chunk 6
Inserted chunk 7
Inserted chunk 8
Inserted chunk 9
Inserted chunk 10
Inserted chunk 11
Inserted chunk 12
Inserted chunk 13
Inserted chunk 14
Inserted chunk 15
Inserted chunk 16
Inserted chunk 17
Inserted chunk 18
Inserted chunk 19
Inserted chunk 20
Inserted chunk 21
Inserted chunk 22
Inserted chunk 23
Inserted chunk 24
Inserted chunk 25
Inserted chunk 26
Inserted chunk 27
Inserted chunk 28
Inserted chunk 29
Inserted chunk 30
Inserted chunk 31
Inserted chunk 32
Inserted chunk 33
Inserted chunk 34
Inserted chunk 35
Inserted chunk 36
Inserted chunk 37
Inserted chunk 38
Inserted chunk 39
Inserted chunk 40
Inserted chunk 41
Inserted chunk 42
Inserted chunk 43
Inserted chunk 44
Inserted chunk 45
Inserted chunk 46
Inserted chunk 47
Inserted chunk 48
Inserted chunk 49
Inserted chunk 50


In [6]:
# Define the seasons and play types
seasons = [
    "2015-16", "2016-17", "2017-18", "2018-19", "2019-20",
    "2020-21", "2021-22", "2022-23", "2023-24", "2024-25"
]

play_types = [
    "Cut", "Handoff", "Isolation", "Misc", "OffScreen",
    "Postup", "PRBallHandler", "PRRollman", "OffRebound", "Spotup", "Transition"
]

groupings = ["offensive", "defensive"]  # Add both

all_synergy_data = []

# Main data pull loop
for season in seasons:
    for grouping in groupings:
        for play_type in play_types:
            for attempt in range(3):
                try:
                    synergy = SynergyPlayTypes(
                        league_id="00",
                        per_mode_simple="PerGame",
                        player_or_team_abbreviation="P",
                        season_type_all_star="Regular Season",
                        season=season,
                        play_type_nullable=play_type,
                        type_grouping_nullable=grouping
                    )
                    
                    df = synergy.get_data_frames()[0]
                    df["SEASON"] = season
                    df["PLAY_TYPE"] = play_type
                    df["TYPE_GROUPING"] = grouping
                    all_synergy_data.append(df)
                    print(f" {grouping} - {play_type} - {season}")
                    break  # success

                except Exception as e:
                    print(f"Attempt {attempt + 1} failed: {grouping} | {play_type} | {season} — {e}")
                    time.sleep(30 + attempt)

            time.sleep(5)

 offensive - Cut - 2015-16
 offensive - Handoff - 2015-16
 offensive - Isolation - 2015-16
 offensive - Misc - 2015-16
 offensive - OffScreen - 2015-16
 offensive - Postup - 2015-16
 offensive - PRBallHandler - 2015-16
 offensive - PRRollman - 2015-16
 offensive - OffRebound - 2015-16
 offensive - Spotup - 2015-16
 offensive - Transition - 2015-16
 defensive - Cut - 2015-16
 defensive - Handoff - 2015-16
 defensive - Isolation - 2015-16
 defensive - Misc - 2015-16
 defensive - OffScreen - 2015-16
 defensive - Postup - 2015-16
 defensive - PRBallHandler - 2015-16
 defensive - PRRollman - 2015-16
 defensive - OffRebound - 2015-16
 defensive - Spotup - 2015-16
 defensive - Transition - 2015-16
 offensive - Cut - 2016-17
 offensive - Handoff - 2016-17
 offensive - Isolation - 2016-17
 offensive - Misc - 2016-17
 offensive - OffScreen - 2016-17
 offensive - Postup - 2016-17
 offensive - PRBallHandler - 2016-17
 offensive - PRRollman - 2016-17
 offensive - OffRebound - 2016-17
 offensive - S

In [8]:
synergy_df = pd.concat(all_synergy_data, ignore_index=True)

synergy_df.to_csv("synergy_play_types_full.csv", index=False)

print(synergy_df)

C:\Users\npowers\AppData\Local\Temp\ipykernel_23564\2300821048.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  synergy_df = pd.concat(all_synergy_data, ignore_index=True)


      SEASON_ID PLAYER_ID      PLAYER_NAME     TEAM_ID TEAM_ABBREVIATION  \
0         22015    101162    Marcin Gortat  1610612764               WAS   
1         22015      2730    Dwight Howard  1610612745               HOU   
2         22015    202324   Derrick Favors  1610612762               UTA   
3         22015    201599   DeAndre Jordan  1610612746               LAC   
4         22015    201572      Brook Lopez  1610612751               BKN   
...         ...       ...              ...         ...               ...   
57936     22024   1630595  Cade Cunningham  1610612765               DET   
57937     22024   1627750     Jamal Murray  1610612743               DEN   
57938     22024   1627783    Pascal Siakam  1610612754               IND   
57939     22024   1631128  Christian Braun  1610612743               DEN   
57940     22024   1628404        Josh Hart  1610612752               NYK   

                TEAM_NAME PLAY_TYPE TYPE_GROUPING  PERCENTILE  GP  ...  \
0      Washin

In [7]:
from nba_api.stats.endpoints import LeagueDashPtStats

pt_types = [
    "SpeedDistance", "Rebounding", "CatchShoot", "PullUpShot", "Drives",
    "Passing", "ElbowTouch", "PostTouch", "PaintTouch", "Efficiency"
]

pt_dfs = {pt: [] for pt in pt_types}  # dict to collect dataframes for each pt type

for season in seasons:
    for pt in pt_types:
        try:
            df = LeagueDashPtStats(
                season=season,
                pt_measure_type=pt,
                player_or_team="Player",
                per_mode_simple="PerGame",
                season_type_all_star="Regular Season"
            ).get_data_frames()[0]

            df["season"] = season
            suffix_cols = [col for col in df.columns if col not in ["PLAYER_ID", "TEAM_ID", "season"]]
            df = df.rename(columns={col: f"{col}_{pt}" for col in suffix_cols})

            pt_dfs[pt].append(df)
            print(f"Pulled: {season} | {pt}")
            time.sleep(1)
        except Exception as e:
            print(f"FAILED: {season} | {pt} --> {e}")
            time.sleep(5)

# Combine all dataframes by merging on PLAYER_ID, TEAM_ID, and season
from functools import reduce
merged_pt_df = reduce(
    lambda left, right: pd.merge(left, right, on=["PLAYER_ID", "TEAM_ID", "season"], how="outer"),
    [pd.concat(pt_dfs[pt], ignore_index=True) for pt in pt_types]
)

# Optional preview
print(merged_pt_df.shape)
print(merged_pt_df.columns.tolist())
print(merged_pt_df)


Pulled: 2015-16 | SpeedDistance
Pulled: 2015-16 | Rebounding
Pulled: 2015-16 | CatchShoot
Pulled: 2015-16 | PullUpShot
Pulled: 2015-16 | Drives
Pulled: 2015-16 | Passing
Pulled: 2015-16 | ElbowTouch
Pulled: 2015-16 | PostTouch
Pulled: 2015-16 | PaintTouch
Pulled: 2015-16 | Efficiency
Pulled: 2016-17 | SpeedDistance
Pulled: 2016-17 | Rebounding
Pulled: 2016-17 | CatchShoot
Pulled: 2016-17 | PullUpShot
Pulled: 2016-17 | Drives
Pulled: 2016-17 | Passing
Pulled: 2016-17 | ElbowTouch
Pulled: 2016-17 | PostTouch
Pulled: 2016-17 | PaintTouch
Pulled: 2016-17 | Efficiency
Pulled: 2017-18 | SpeedDistance
Pulled: 2017-18 | Rebounding
Pulled: 2017-18 | CatchShoot
Pulled: 2017-18 | PullUpShot
Pulled: 2017-18 | Drives
Pulled: 2017-18 | Passing
Pulled: 2017-18 | ElbowTouch
Pulled: 2017-18 | PostTouch
Pulled: 2017-18 | PaintTouch
Pulled: 2017-18 | Efficiency
Pulled: 2018-19 | SpeedDistance
Pulled: 2018-19 | Rebounding
Pulled: 2018-19 | CatchShoot
Pulled: 2018-19 | PullUpShot
Pulled: 2018-19 | Drives
P

In [10]:
print(synergy_df.columns.tolist())
print(merged_pt_df.columns.tolist())




['SEASON_ID', 'PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'PLAY_TYPE', 'TYPE_GROUPING', 'PERCENTILE', 'GP', 'POSS_PCT', 'PPP', 'FG_PCT', 'FT_POSS_PCT', 'TOV_POSS_PCT', 'SF_POSS_PCT', 'PLUSONE_POSS_PCT', 'SCORE_POSS_PCT', 'EFG_PCT', 'POSS', 'PTS', 'FGM', 'FGA', 'FGMX', 'SEASON']
['PLAYER_ID', 'PLAYER_NAME_SpeedDistance', 'TEAM_ID', 'TEAM_ABBREVIATION_SpeedDistance', 'GP_SpeedDistance', 'W_SpeedDistance', 'L_SpeedDistance', 'MIN_SpeedDistance', 'MIN1_SpeedDistance', 'DIST_FEET_SpeedDistance', 'DIST_MILES_SpeedDistance', 'DIST_MILES_OFF_SpeedDistance', 'DIST_MILES_DEF_SpeedDistance', 'AVG_SPEED_SpeedDistance', 'AVG_SPEED_OFF_SpeedDistance', 'AVG_SPEED_DEF_SpeedDistance', 'season', 'PLAYER_NAME_Rebounding', 'TEAM_ABBREVIATION_Rebounding', 'GP_Rebounding', 'W_Rebounding', 'L_Rebounding', 'MIN_Rebounding', 'OREB_Rebounding', 'OREB_CONTEST_Rebounding', 'OREB_UNCONTEST_Rebounding', 'OREB_CONTEST_PCT_Rebounding', 'OREB_CHANCES_Rebounding', 'OREB_CHANCE_PCT_Reboundin

In [6]:
synergy_df = pd.read_csv("synergy_play_types_full.csv")

In [8]:
synergy_selected_columns = ['SEASON_ID', 'PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID', 'PLAY_TYPE', 'TYPE_GROUPING', 'PERCENTILE', 'POSS_PCT', 'PPP', 'FG_PCT', 'FT_POSS_PCT', 'TOV_POSS_PCT', 'SF_POSS_PCT', 'PLUSONE_POSS_PCT', 'SCORE_POSS_PCT', 'EFG_PCT', 'POSS', 'PTS', 'FGM', 'FGA', 'FGMX', 'SEASON']
synergy_filtered_df = synergy_df[synergy_selected_columns]


print(synergy_filtered_df)


       SEASON_ID  PLAYER_ID      PLAYER_NAME     TEAM_ID PLAY_TYPE  \
0          22015     101162    Marcin Gortat  1610612764       Cut   
1          22015       2730    Dwight Howard  1610612745       Cut   
2          22015     202324   Derrick Favors  1610612762       Cut   
3          22015     201599   DeAndre Jordan  1610612746       Cut   
4          22015     201572      Brook Lopez  1610612751       Cut   
...          ...        ...              ...         ...       ...   
57936      22024    1630595  Cade Cunningham  1610612765    Spotup   
57937      22024    1627750     Jamal Murray  1610612743    Spotup   
57938      22024    1627783    Pascal Siakam  1610612754    Spotup   
57939      22024    1631128  Christian Braun  1610612743    Spotup   
57940      22024    1628404        Josh Hart  1610612752    Spotup   

      TYPE_GROUPING  PERCENTILE  POSS_PCT    PPP  FG_PCT  ...  SF_POSS_PCT  \
0         offensive       0.353     0.314  1.134   0.577  ...        0.104   
1  

In [ ]:
playtype_selected_columns = ['PLAYER_ID','season', 'PLAYER_NAME_SpeedDistance', 'TEAM_ID',
                             
                            'DIST_MILES_OFF_SpeedDistance','DIST_MILES_DEF_SpeedDistance', 'AVG_SPEED_SpeedDistance', 
                            'AVG_SPEED_OFF_SpeedDistance', 'AVG_SPEED_DEF_SpeedDistance', # Speed and Distance Group

                            'OREB_Rebounding', 'OREB_CONTEST_Rebounding', 'OREB_CONTEST_PCT_Rebounding',
                            'OREB_CHANCES_Rebounding', 'OREB_CHANCE_PCT_Rebounding', 
                            'OREB_UNCONTEST_Rebounding', 'AVG_OREB_DIST_Rebounding', # Offensive Rebounding Group
                            
                            'DREB_Rebounding', 'DREB_CONTEST_Rebounding', 'DREB_CONTEST_PCT_Rebounding',
                            'DREB_CHANCES_Rebounding', 'DREB_CHANCE_PCT_Rebounding',
                            'DREB_UNCONTEST_Rebounding', 'AVG_DREB_DIST_Rebounding', # Defensive Rebounding Group

                            'CATCH_SHOOT_FGM_CatchShoot','CATCH_SHOOT_FGA_CatchShoot', 'CATCH_SHOOT_FG_PCT_CatchShoot', 
                            'CATCH_SHOOT_FG3M_CatchShoot', 'CATCH_SHOOT_FG3A_CatchShoot', 'CATCH_SHOOT_FG3_PCT_CatchShoot',
                            'CATCH_SHOOT_PTS_CatchShoot', 'CATCH_SHOOT_EFG_PCT_CatchShoot', # Catch and Shoot Group


                            'PULL_UP_FGM_PullUpShot', 'PULL_UP_FGA_PullUpShot', 'PULL_UP_FG_PCT_PullUpShot', 
                            'PULL_UP_FG3M_PullUpShot', 'PULL_UP_FG3A_PullUpShot', 'PULL_UP_FG3_PCT_PullUpShot', # Pull Up Shot Group
                            'PULL_UP_PTS_PullUpShot','PULL_UP_EFG_PCT_PullUpShot', 


                            'DRIVES_Drives', 'DRIVE_PTS_Drives', 'DRIVE_PTS_PCT_Drives',
                            'DRIVE_FGM_Drives', 'DRIVE_FGA_Drives', 'DRIVE_FG_PCT_Drives',
                            'DRIVE_FTA_Drives',
                            'DRIVE_PASSES_PCT_Drives', 'DRIVE_AST_PCT_Drives',
                            'DRIVE_TOV_PCT_Drives', 'DRIVE_PF_PCT_Drives', # Drives Group


                            'PASSES_MADE_Passing', 'PASSES_RECEIVED_Passing', 
                            'AST_Passing', 'SECONDARY_AST_Passing', 'POTENTIAL_AST_Passing', 
                            'FT_AST_Passing',
                            'AST_POINTS_CREATED_Passing', 'AST_ADJ_Passing', 'AST_TO_PASS_PCT_Passing', 
                            # Passing Group


                            'ELBOW_TOUCHES_ElbowTouch', 'ELBOW_TOUCH_PTS_ElbowTouch', 'ELBOW_TOUCH_PTS_PCT_ElbowTouch',
                            'ELBOW_TOUCH_FGM_ElbowTouch', 'ELBOW_TOUCH_FGA_ElbowTouch', 'ELBOW_TOUCH_FG_PCT_ElbowTouch', 
                            'ELBOW_TOUCH_FTA_ElbowTouch', 
                            'ELBOW_TOUCH_AST_PCT_ElbowTouch', 'ELBOW_TOUCH_PASSES_PCT_ElbowTouch',
                            'ELBOW_TOUCH_TOV_PCT_ElbowTouch', 
                            # Elbow Touch Group
                            

                            'POST_TOUCHES_PostTouch', 'POST_TOUCH_PTS_PostTouch', 'POST_TOUCH_PTS_PCT_PostTouch',
                            'POST_TOUCH_FGM_PostTouch', 'POST_TOUCH_FGA_PostTouch', 'POST_TOUCH_FG_PCT_PostTouch', 
                            'POST_TOUCH_FTA_PostTouch',
                            'POST_TOUCH_AST_PCT_PostTouch','POST_TOUCH_PASSES_PCT_PostTouch',
                            'POST_TOUCH_TOV_PCT_PostTouch', 
                            # Post Touch Group

                            'PAINT_TOUCHES_PaintTouch','PAINT_TOUCH_PTS_PaintTouch', 'PAINT_TOUCH_PTS_PCT_PaintTouch',  
                            'PAINT_TOUCH_FGM_PaintTouch', 'PAINT_TOUCH_FGA_PaintTouch', 'PAINT_TOUCH_FG_PCT_PaintTouch', 
                            'PAINT_TOUCH_FTA_PaintTouch', 
                            'PAINT_TOUCH_AST_PCT_PaintTouch','PAINT_TOUCH_PASSES_PCT_PaintTouch', 
                            'PAINT_TOUCH_TOV_PCT_PaintTouch'] # Paint Touch Group


playtype_values_df = merged_pt_df[playtype_selected_columns]

In [27]:
playtype_values_df = playtype_values_df.rename(columns={"PLAYER_NAME_SpeedDistance": "PLAYER_NAME"})

# Define columns to keep as grouping keys (non-numeric)
group_keys = ['PLAYER_ID', 'season', 'PLAYER_NAME', 'TEAM_ID']

# Identify numeric columns to convert to percentiles
numeric_cols = [col for col in playtype_values_df.columns if col not in group_keys]

# Create percentile dataframe
playtype_percentile_df = playtype_values_df[group_keys].copy()

# Calculate percentiles by season
for col in numeric_cols:
    playtype_percentile_df[col + "_pct"] = playtype_values_df.groupby("season")[col].rank(pct=True)

# Invert percentiles for turnover-related columns
for col in numeric_cols:
    if "TOV" in col.upper():
        pct_col = col + "_pct"
        playtype_percentile_df[pct_col] = 1 - playtype_percentile_df[pct_col]


# Preview
print(playtype_percentile_df)


      PLAYER_ID   season      PLAYER_NAME     TEAM_ID  \
0        201166  2015-16     Aaron Brooks  1610612741   
1        203932  2015-16     Aaron Gordon  1610612753   
2       1626151  2015-16   Aaron Harrison  1610612766   
3        203940  2015-16    Adreian Payne  1610612750   
4        201143  2015-16       Al Horford  1610612737   
...         ...      ...              ...         ...   
5381     203897  2024-25      Zach LaVine  1610612758   
5382    1630192  2024-25       Zeke Nnaji  1610612743   
5383    1630533  2024-25  Ziaire Williams  1610612751   
5384    1629627  2024-25  Zion Williamson  1610612740   
5385    1642389  2024-25      Zyon Pullin  1610612763   

      DIST_MILES_OFF_SpeedDistance_pct  DIST_MILES_DEF_SpeedDistance_pct  \
0                             0.330882                          0.338235   
1                             0.707983                          0.678571   
2                             0.030462                          0.045168   
3          

In [28]:
playtype_merged_df = playtype_values_df.merge(
    team_roster_df[["player_id", "season", "position", "age", "experience"]],
    left_on=["PLAYER_ID", "season"],
    right_on=["player_id", "season"],
    how="left"
)

playtype_merged_df = playtype_merged_df.dropna(subset=["player_id"])
playtype_merged_df = playtype_merged_df.drop_duplicates()

playtype_merged_df = playtype_merged_df.drop(columns=["player_id"], errors="ignore")

print(playtype_merged_df)

      PLAYER_ID   season      PLAYER_NAME     TEAM_ID  \
0        201166  2015-16     Aaron Brooks  1610612741   
1        203932  2015-16     Aaron Gordon  1610612753   
2       1626151  2015-16   Aaron Harrison  1610612766   
3        203940  2015-16    Adreian Payne  1610612750   
4        201143  2015-16       Al Horford  1610612737   
...         ...      ...              ...         ...   
5382     203897  2024-25      Zach LaVine  1610612758   
5383    1630192  2024-25       Zeke Nnaji  1610612743   
5384    1630533  2024-25  Ziaire Williams  1610612751   
5385    1629627  2024-25  Zion Williamson  1610612740   
5386    1642389  2024-25      Zyon Pullin  1610612763   

      DIST_MILES_OFF_SpeedDistance  DIST_MILES_DEF_SpeedDistance  \
0                             0.58                          0.51   
1                             1.00                          0.83   
2                             0.17                          0.16   
3                             0.36         

In [10]:
synergy_merged_df = synergy_filtered_df.merge(
    team_roster_df[["player_id", "season", "position", "age", "experience"]],
    left_on=["PLAYER_ID", "SEASON"],
    right_on=["player_id", "season"],
    how="left"
)

synergy_merged_df = synergy_merged_df.dropna(subset=["player_id"])
synergy_merged_df = synergy_merged_df.drop_duplicates()

synergy_merged_df = synergy_merged_df.drop(columns=["player_id","season"], errors="ignore")

print(synergy_merged_df.columns)

Index(['SEASON_ID', 'PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID', 'PLAY_TYPE',
       'TYPE_GROUPING', 'PERCENTILE', 'POSS_PCT', 'PPP', 'FG_PCT',
       'FT_POSS_PCT', 'TOV_POSS_PCT', 'SF_POSS_PCT', 'PLUSONE_POSS_PCT',
       'SCORE_POSS_PCT', 'EFG_PCT', 'POSS', 'PTS', 'FGM', 'FGA', 'FGMX',
       'SEASON', 'position', 'age', 'experience'],
      dtype='object')


In [53]:
# Define columns to keep as grouping keys (non-numeric)
group_keys = ['PLAYER_ID', 'age', 'season', 'PLAYER_NAME', 'TEAM_ID']

# Identify numeric columns to convert to percentiles
numeric_cols = [col for col in playtype_merged_df.columns if col not in group_keys]

# Create percentile dataframe
playtype_percentile_by_age_df = playtype_merged_df[group_keys].copy()

# Calculate percentiles by age
for col in numeric_cols:
    playtype_percentile_by_age_df[col + "_pct"] = playtype_merged_df.groupby("age")[col].rank(pct=True)

# Invert percentiles for turnover-related columns
for col in numeric_cols:
    if "TOV" in col.upper():
        pct_col = col + "_pct"
        playtype_percentile_by_age_df[pct_col] = 1 - playtype_percentile_by_age_df[pct_col]

print(playtype_percentile_by_age_df)


      PLAYER_ID   age   season      PLAYER_NAME     TEAM_ID  \
0        201166  31.0  2015-16     Aaron Brooks  1610612741   
1        203932  20.0  2015-16     Aaron Gordon  1610612753   
2       1626151  21.0  2015-16   Aaron Harrison  1610612766   
3        203940  25.0  2015-16    Adreian Payne  1610612750   
4        201143  30.0  2015-16       Al Horford  1610612737   
...         ...   ...      ...              ...         ...   
5382     203897  30.0  2024-25      Zach LaVine  1610612758   
5383    1630192  24.0  2024-25       Zeke Nnaji  1610612743   
5384    1630533  23.0  2024-25  Ziaire Williams  1610612751   
5385    1629627  24.0  2024-25  Zion Williamson  1610612740   
5386    1642389  24.0  2024-25      Zyon Pullin  1610612763   

      DIST_MILES_OFF_SpeedDistance_pct  DIST_MILES_DEF_SpeedDistance_pct  \
0                             0.186603                          0.172249   
1                             0.735981                          0.719626   
2              

In [42]:
# Define columns to keep as grouping keys (non-numeric)
group_keys = ['PLAYER_ID', 'experience' , 'season' , 'PLAYER_NAME', 'TEAM_ID']

# Identify numeric columns to convert to percentiles
numeric_cols = [col for col in playtype_merged_df.columns if col not in group_keys]

# Create percentile dataframe
playtype_percentile_by_exp_df = playtype_merged_df[group_keys].copy()

# Calculate percentiles by experience
for col in numeric_cols:
    playtype_percentile_by_exp_df[col + "_pct"] = playtype_merged_df.groupby("experience")[col].rank(pct=True)

# Invert percentiles for turnover-related columns
for col in numeric_cols:
    if "TOV" in col.upper():
        pct_col = col + "_pct"
        playtype_percentile_by_exp_df[pct_col] = 1 - playtype_percentile_by_exp_df[pct_col]


print(playtype_percentile_by_exp_df)


      PLAYER_ID  experience   season      PLAYER_NAME     TEAM_ID  \
0        201166         7.0  2015-16     Aaron Brooks  1610612741   
1        203932         1.0  2015-16     Aaron Gordon  1610612753   
2       1626151         0.0  2015-16   Aaron Harrison  1610612766   
3        203940         1.0  2015-16    Adreian Payne  1610612750   
4        201143         8.0  2015-16       Al Horford  1610612737   
...         ...         ...      ...              ...         ...   
5382     203897        10.0  2024-25      Zach LaVine  1610612758   
5383    1630192         4.0  2024-25       Zeke Nnaji  1610612743   
5384    1630533         3.0  2024-25  Ziaire Williams  1610612751   
5385    1629627         4.0  2024-25  Zion Williamson  1610612740   
5386    1642389         0.0  2024-25      Zyon Pullin  1610612763   

      DIST_MILES_OFF_SpeedDistance_pct  DIST_MILES_DEF_SpeedDistance_pct  \
0                             0.138672                          0.144531   
1                  

In [43]:
# Guard, Wing, Forward definitions
guard_positions = ["G", "G-F"]
wing_positions = ["F", "F-G"]
forward_positions = ["C", "C-F", "F-C"]


In [44]:
guards_df = playtype_merged_df[playtype_merged_df["position"].isin(guard_positions)]
wings_df = playtype_merged_df[playtype_merged_df["position"].isin(wing_positions)]
forwards_df = playtype_merged_df[playtype_merged_df["position"].isin(forward_positions)]


In [45]:
def calculate_percentiles_by_position_group(df):
    group_keys = ['PLAYER_ID', 'position', 'season', 'PLAYER_NAME', 'TEAM_ID']
    numeric_cols = [col for col in df.columns if col not in group_keys]
    
    percentile_df = df[group_keys].copy()
    
    for col in numeric_cols:
        percentile_df[col + "_pct"] = df[col].rank(pct=True)
        
    for col in numeric_cols:
        if "TOV" in col.upper():
            pct_col = col + "_pct"
            percentile_df[pct_col] = 1 - percentile_df[pct_col]
    
    return percentile_df


In [46]:
guard_percentiles = calculate_percentiles_by_position_group(guards_df)
wing_percentiles = calculate_percentiles_by_position_group(wings_df)
forward_percentiles = calculate_percentiles_by_position_group(forwards_df)


In [11]:
synergy_merged_df["PPP_pct_by_age"] = synergy_merged_df.groupby("age")["PPP"].rank(pct=True)
synergy_merged_df["PPP_pct_by_experience"] = synergy_merged_df.groupby("experience")["PPP"].rank(pct=True)

In [12]:
# Define position groups
guard_positions = ["G", "G-F"]
wing_positions = ["F", "F-G"]
forward_positions = ["C", "C-F", "F-C"]

# Create a new column for position group
def get_position_group(pos):
    if pos in guard_positions:
        return "Guard"
    elif pos in wing_positions:
        return "Wing"
    elif pos in forward_positions:
        return "Forward"
    else:
        return "Other"

synergy_merged_df["position_group"] = synergy_merged_df["position"].apply(get_position_group)

# Calculate PPP percentiles by position group
synergy_merged_df["PPP_pct_by_position_group"] = synergy_merged_df.groupby("position_group")["PPP"].rank(pct=True)

print(synergy_merged_df)


       SEASON_ID  PLAYER_ID      PLAYER_NAME     TEAM_ID PLAY_TYPE  \
0          22015     101162    Marcin Gortat  1610612764       Cut   
1          22015       2730    Dwight Howard  1610612745       Cut   
2          22015     202324   Derrick Favors  1610612762       Cut   
3          22015     201599   DeAndre Jordan  1610612746       Cut   
4          22015     201572      Brook Lopez  1610612751       Cut   
...          ...        ...              ...         ...       ...   
57940      22024    1630595  Cade Cunningham  1610612765    Spotup   
57941      22024    1627750     Jamal Murray  1610612743    Spotup   
57942      22024    1627783    Pascal Siakam  1610612754    Spotup   
57943      22024    1631128  Christian Braun  1610612743    Spotup   
57944      22024    1628404        Josh Hart  1610612752    Spotup   

      TYPE_GROUPING  PERCENTILE  POSS_PCT    PPP  FG_PCT  ...  FGA  FGMX  \
0         offensive       0.353     0.314  1.134   0.577  ...  3.5   1.5   
1      

In [ ]:
for col in ["PPP_pct_by_age", "PPP_pct_by_experience", "PPP_pct_by_position_group"]:
    synergy_merged_df.loc[synergy_merged_df["TYPE_GROUPING"] == "defensive", col] = (
        1 - synergy_merged_df.loc[synergy_merged_df["TYPE_GROUPING"] == "defensive", col]
    )

print(synergy_merged_df)

In [9]:
# Create SQL to define your table schema
def generate_supabase_sql(df, table_name):
    sql_cols = []
    for col in df.columns:
        dtype = df[col].dtype
        if pd.api.types.is_integer_dtype(dtype):
            sql_cols.append(f'"{col}" bigint')
        elif pd.api.types.is_float_dtype(dtype):
            sql_cols.append(f'"{col}" double precision')
        else:
            sql_cols.append(f'"{col}" text')  # text for object, str, etc.

    sql_statement = f"CREATE TABLE {table_name} (\n  " + ",\n  ".join(sql_cols) + "\n);"
    return sql_statement



In [ ]:
sql_create = generate_supabase_sql(playtype_values_df, "playtype_values")
print(sql_create) 

sql_create = generate_supabase_sql(playtype_percentile_df, "playtype_percentiles")
print(sql_create) 

sql_create = generate_supabase_sql(playtype_percentile_by_age_df, "playtype_percentiles_age")
print(sql_create) 

sql_create = generate_supabase_sql(playtype_percentile_by_exp_df, "playtype_percentiles_exp")
print(sql_create) 

sql_create = generate_supabase_sql(guard_percentiles, "guard_percentiles")
print(sql_create) 

sql_create = generate_supabase_sql(wing_percentiles, "wing_percentiles")
print(sql_create) 

sql_create = generate_supabase_sql(forward_percentiles, "forward_percentiles")
print(sql_create) 







CREATE TABLE synergy (
  "SEASON_ID" bigint,
  "PLAYER_ID" bigint,
  "PLAYER_NAME" text,
  "TEAM_ID" bigint,
  "PLAY_TYPE" text,
  "TYPE_GROUPING" text,
  "PERCENTILE" double precision,
  "POSS_PCT" double precision,
  "PPP" double precision,
  "FG_PCT" double precision,
  "FT_POSS_PCT" double precision,
  "TOV_POSS_PCT" double precision,
  "SF_POSS_PCT" double precision,
  "PLUSONE_POSS_PCT" double precision,
  "SCORE_POSS_PCT" double precision,
  "EFG_PCT" double precision,
  "POSS" double precision,
  "PTS" double precision,
  "FGM" double precision,
  "FGA" double precision,
  "FGMX" double precision,
  "SEASON" text,
  "season" text,
  "position" text,
  "age" double precision,
  "experience" double precision,
  "PPP_pct_by_age" double precision,
  "PPP_pct_by_experience" double precision,
  "position_group" text,
  "PPP_pct_by_position_group" double precision
);
CREATE TABLE playtype_values (
  "PLAYER_ID" bigint,
  "season" text,
  "PLAYER_NAME" text,
  "TEAM_ID" bigint,
  "DIST

In [10]:
sql_create = generate_supabase_sql(team_roster_df, "teams_players")
print(sql_create) 

CREATE TABLE teams_players (
  "season" text,
  "team_id" bigint,
  "player_id" bigint,
  "player_name" text,
  "position" text,
  "age" double precision,
  "experience" bigint
);


In [16]:
records = synergy_merged_df.to_dict(orient="records")
chunk_size = 1000
for i in range(0, len(records), chunk_size):
    chunk = records[i:i + chunk_size]
    try:
        supabase.table("synergy").insert(chunk).execute()
        print(f"Inserted chunk {i // chunk_size + 1}")
    except Exception as e:
        print(f"Error inserting chunk {i // chunk_size + 1}: {e}")


Inserted chunk 1
Inserted chunk 2
Inserted chunk 3
Inserted chunk 4
Inserted chunk 5
Inserted chunk 6
Inserted chunk 7
Inserted chunk 8
Inserted chunk 9
Inserted chunk 10
Inserted chunk 11
Inserted chunk 12
Inserted chunk 13
Inserted chunk 14
Inserted chunk 15
Inserted chunk 16
Inserted chunk 17
Inserted chunk 18
Inserted chunk 19
Inserted chunk 20
Inserted chunk 21
Inserted chunk 22
Inserted chunk 23
Inserted chunk 24
Inserted chunk 25
Inserted chunk 26
Inserted chunk 27
Inserted chunk 28
Inserted chunk 29
Inserted chunk 30
Inserted chunk 31
Inserted chunk 32
Inserted chunk 33
Inserted chunk 34
Inserted chunk 35
Inserted chunk 36
Inserted chunk 37
Inserted chunk 38
Inserted chunk 39
Inserted chunk 40
Inserted chunk 41
Inserted chunk 42
Inserted chunk 43
Inserted chunk 44
Inserted chunk 45
Inserted chunk 46
Inserted chunk 47
Inserted chunk 48
Inserted chunk 49
Inserted chunk 50
Inserted chunk 51
Inserted chunk 52
Inserted chunk 53
Inserted chunk 54
Inserted chunk 55
Inserted chunk 56
I

In [11]:
records = team_roster_df.to_dict(orient="records")
chunk_size = 1000
for i in range(0, len(records), chunk_size):
    chunk = records[i:i + chunk_size]
    try:
        supabase.table("teams_players").insert(chunk).execute()
        print(f"Inserted chunk {i // chunk_size + 1}")
    except Exception as e:
        print(f"Error inserting chunk {i // chunk_size + 1}: {e}")


Inserted chunk 1
Inserted chunk 2
Inserted chunk 3
Inserted chunk 4
Inserted chunk 5


In [52]:
# Replace inf and -inf with NaN first
playtype_values_df_cleaned = playtype_values_df.replace([np.inf, -np.inf], np.nan)

# Replace all NaNs with 0
playtype_values_df_cleaned = playtype_values_df_cleaned.fillna(0)


records = playtype_values_df_cleaned.to_dict(orient="records")
chunk_size = 1000
for i in range(0, len(records), chunk_size):
    chunk = records[i:i + chunk_size]
    try:
        supabase.table("playtype_values").insert(chunk).execute()
        print(f"Inserted chunk {i // chunk_size + 1}")
    except Exception as e:
        print(f"Error inserting chunk {i // chunk_size + 1}: {e}")



Inserted chunk 1
Inserted chunk 2
Inserted chunk 3
Inserted chunk 4
Inserted chunk 5
Inserted chunk 6


In [54]:
# Clean playtype_percentile_df
playtype_percentile_df_cleaned = playtype_percentile_df.replace([np.inf, -np.inf], np.nan)
playtype_percentile_df_cleaned = playtype_percentile_df_cleaned.fillna(0)

records = playtype_percentile_df_cleaned.to_dict(orient="records")
chunk_size = 1000
for i in range(0, len(records), chunk_size):
    chunk = records[i:i + chunk_size]
    try:
        supabase.table("playtype_percentiles").insert(chunk).execute()
        print(f"Inserted chunk {i // chunk_size + 1}")
    except Exception as e:
        print(f"Error inserting chunk {i // chunk_size + 1}: {e}")

Inserted chunk 1
Inserted chunk 2
Inserted chunk 3
Inserted chunk 4
Inserted chunk 5
Inserted chunk 6


In [51]:
# Clean playtype_percentile_df
playtype_percentile_by_age_df_cleaned = playtype_percentile_by_age_df.replace([np.inf, -np.inf], np.nan)
playtype_percentile_by_age_df_cleaned = playtype_percentile_by_age_df_cleaned.fillna(0)

records = playtype_percentile_by_age_df_cleaned.to_dict(orient="records")
chunk_size = 1000
for i in range(0, len(records), chunk_size):
    time.sleep(5)
    chunk = records[i:i + chunk_size]
    try:
        supabase.table("playtype_percentiles_age").insert(chunk).execute()
        print(f"Inserted chunk {i // chunk_size + 1}")
    except Exception as e:
        print(f"Error inserting chunk {i // chunk_size + 1}: {e}")

Inserted chunk 1
Inserted chunk 2
Inserted chunk 3
Inserted chunk 4
Inserted chunk 5


In [52]:
# Clean playtype_percentile_df
playtype_percentile_by_exp_df_cleaned = playtype_percentile_by_exp_df.replace([np.inf, -np.inf], np.nan)
playtype_percentile_by_exp_df_cleaned = playtype_percentile_by_exp_df_cleaned.fillna(0)

records = playtype_percentile_by_exp_df_cleaned.to_dict(orient="records")
chunk_size = 1000
for i in range(0, len(records), chunk_size):
    time.sleep(2)
    chunk = records[i:i + chunk_size]
    try:
        supabase.table("playtype_percentiles_exp").insert(chunk).execute()
        print(f"Inserted chunk {i // chunk_size + 1}")
    except Exception as e:
        print(f"Error inserting chunk {i // chunk_size + 1}: {e}")

Inserted chunk 1
Inserted chunk 2
Inserted chunk 3
Inserted chunk 4
Inserted chunk 5


In [49]:
# Clean playtype_percentile_df
guard_percentiles_cleaned = guard_percentiles.replace([np.inf, -np.inf], np.nan)
guard_percentiles_cleaned = guard_percentiles_cleaned.fillna(0)

records = guard_percentiles_cleaned.to_dict(orient="records")
chunk_size = 1000
for i in range(0, len(records), chunk_size):
    time.sleep(2)
    chunk = records[i:i + chunk_size]
    try:
        supabase.table("guard_percentiles").insert(chunk).execute()
        print(f"Inserted chunk {i // chunk_size + 1}")
    except Exception as e:
        print(f"Error inserting chunk {i // chunk_size + 1}: {e}")

Inserted chunk 1
Inserted chunk 2
Inserted chunk 3


In [50]:
# Clean playtype_percentile_df
wing_percentiles_cleaned = wing_percentiles.replace([np.inf, -np.inf], np.nan)
wing_percentiles_cleaned = wing_percentiles_cleaned.fillna(0)

records = wing_percentiles_cleaned.to_dict(orient="records")
chunk_size = 1000
for i in range(0, len(records), chunk_size):
    time.sleep(2)
    chunk = records[i:i + chunk_size]
    try:
        supabase.table("wing_percentiles").insert(chunk).execute()
        print(f"Inserted chunk {i // chunk_size + 1}")
    except Exception as e:
        print(f"Error inserting chunk {i // chunk_size + 1}: {e}")

Inserted chunk 1
Inserted chunk 2


In [48]:
# Clean playtype_percentile_df
forward_percentiles_cleaned = forward_percentiles.replace([np.inf, -np.inf], np.nan)
forward_percentiles_cleaned = forward_percentiles_cleaned.fillna(0)

records = forward_percentiles_cleaned.to_dict(orient="records")
chunk_size = 1000
for i in range(0, len(records), chunk_size):
    time.sleep(2)
    chunk = records[i:i + chunk_size]
    try:
        supabase.table("forward_percentiles").insert(chunk).execute()
        print(f"Inserted chunk {i // chunk_size + 1}")
    except Exception as e:
        print(f"Error inserting chunk {i // chunk_size + 1}: {e}")

Inserted chunk 1
